# Introducción a SpaCy

[Pablo Carballeira] Partes de este código han sido adaptadas del código de Victor Peinado https://github.com/vitojph/kschool-nlp-23

Puedes encontrar información sobre cómo trabajar en Colab aquí (https://colab.research.google.com/notebooks/intro.ipynb)






In [ ]:
# install the requirements
# !pip install spacy
!pip install -U spacy
# !python -m spacy download es_core_news_md
!python -m spacy download es_core_news_lg
!python -m spacy download en_core_web_sm
!python -m spacy validate

[spacy](http://www.spacy.io/) es una librería de procesamiento del lenguaje natural, robusta, rápida, fácil de instalar y utilizar e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/usage/facts-figures#section-other-libraries). spaCy está diseñado específicamente para uso en producción y facilita la creación de aplicaciones que procesan y “entienden” grandes volúmenes de texto. Se puede usar para construir sistemas de extracción de información o comprensión del lenguaje natural, o para preprocesar texto para algoritmos de aprendizaje.




Tiene modelos entrenados en varios idiomas y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión.

Una vez instalados los modelos, podemos importarlos fácilmente:

In [ ]:
import spacy

# cargamos el modelo entrenado en español
nlp_es = spacy.load('es_core_news_lg')

In [ ]:
texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""

# y procesamos el texto
doc = nlp_es(texto)
print(type(doc))

## Procesando oraciones, palabras y entidades

Podemos iterar fácilmente sobre la lista de oraciones y recorrer los tokens para acceder a su información morfo-sintáctica:

Una vez que hemos procesado el texto con el modelo, obtenemos un objeto [doc](https://spacy.io/api/doc), que, formado por oraciones de [tokens](https://spacy.io/api/token#attributes) que contienen muchos atributos generados a partir del análisis de texto a los que podemos acceder facilmente. 

In [ ]:
for sentence in doc.sents:
    print("Oración: {}".format(sentence))
    for token in sentence:
        print(
            "{}/{} => etiqueta {}, y dependencia {} de \"{}\"".format(
                token, token.lemma_, token.pos_, token.dep_, token.head.text
            )
        )

## Análisis sintáctico

El análisis sintáctico es una forma de determinar la estructura de una oración. Es un paso útil para mejorar el rendimiento en aplicaciones como análisis de opinión, predicción de lenguaje, traducción automática, chatbots, etc. Se puede utilizar, por ejemplo para refinar el resultado de clasificadores de texto para análisis de opinión:

Apple was doing poorly until Steve Jobs…

Because Apple was doing poorly, Steve Jobs…

Apple was doing poorly because Steve Jobs…

- En la primera oración, Apple tiene carácter negativo, mientras que Steve Jobs tiene carácter positivo.
- En la segunda, Apple sigue teniendo carácter negativo, pero Steve Jobs ahora tiene carácter neutral.
- En el ejemplo final, tanto Apple como Steve Jobs tienen carácter negativo.

In [ ]:
from nltk.tree import Tree

def tok_format(tok):
    return "_".join([tok.orth_, tok.tag_, tok.dep_])

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(tok_format(node), [to_nltk_tree(child) for child in node.children])
    else:
        return tok_format(node)

ejemplo = "Mi perro persigue a su hijo en la bicicleta pero mi perro no sabe andar en bicicleta"
es_doc = nlp_es(ejemplo) 

[to_nltk_tree(sent.root).pretty_print() for sent in es_doc.sents]

## Visualizando árboles de dependencias y entidades

Podemos dibujar el grafo con las dependencias:

In [ ]:
doc = nlp_es('Madrid es la capital de España.')

In [ ]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

## Reconocimiento de entidades

El reconocimiento de entidades (Named Entity Recognition) es el proceso de identificar sustantivos relevantes (personas, lugares y organizaciones) que se mencionan en una cadena de texto. Esto tiene múltiples aplicaciones. Aquí algunos ejemplos: 

- Clasificación del contenido para los proveedores de noticias: Permite escanear automáticamente artículos completos y ayudar a identificar y recuperar personas, organizaciones y lugares importantes discutidos en ellos. Por lo tanto, los artículos se clasifican automáticamente en jerarquías definidas y se puede navegar por el contenido mucho más fácilmente.

- Resumen automático de currículums: Una de las tareas desafiantes que enfrentan los departamentos de recursos humanos en todas las empresas es evaluar una pila gigantesca de currículos para preseleccionar candidatos. NER permite extraer la información relevante para el evaluador se puede recuperar fácilmente de ellos, lo que simplifica el esfuerzo requerido para preseleccionar candidatos entre una pila de currículos.

- Optimización de algoritmos de motores de búsqueda: al diseñar un algoritmo de motor de búsqueda, es ineficiente buscar una consulta completa en los millones de artículos y sitios web en línea, una forma alternativa es ejecutar un modelo NER en los artículos una vez y almacenar las entidades asociadas a ellos de forma permanente. Por lo tanto, para una búsqueda rápida y eficiente, las etiquetas clave en la consulta de búsqueda se pueden comparar con las etiquetas asociadas con los artículos del sitio web.

- Simplificar la atención al cliente: Una empresa recibe toneladas de quejas y comentarios de los clientes a diario, y revisar cada uno de ellos y reconocer a las partes involucradas no es una tarea fácil. Con NER, podemos reconocer entidades relevantes en las quejas y comentarios de los clientes, como productos, o la ubicación de la sucursal de la empresa, de modo que los comentarios se pueden clasificar y se envían al departamento correspondiente.

...


El elemento `doc` tiene una propiedad `.ents` que permite acceder a las entidades que hayan sido localizadas en un texto

In [ ]:
texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""
doc = nlp_es(texto)

for entity in doc.ents:
    print("{} es de tipo {}".format(entity, entity.label_))

También ofrece herramientas para extraer trozos de texto que corresponden a sintagmas nominales

In [ ]:
nlp_en = spacy.load("en_core_web_sm")
doc = nlp_en("Autonomous cars shift insurance liability toward manufacturers")
for chunk in doc.noun_chunks:
    print("Noun sent: ", chunk.text, ", Noun: ", chunk.root.text,", Dep: " , chunk.root.dep_, ", Root: ",
            chunk.root.head.text)

In [ ]:
doc2 = nlp_es(u'Me gustó mucho trabajar en Repsol con mi compañero Richard Thompson')

In [ ]:
for ent in doc2.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
displacy.render(doc2, style='ent', jupyter=True)

In [ ]:
texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""
doc = nlp_es(texto)
displacy.render(doc, style="ent", jupyter=True)

## Ejemplo de aplicación:

Presentamos aquí un ejemplo de aplicación práctica que podemos tener, e forma sencilla, combinando varias de las herramientas que nos ofrece SpaCy. 

En el ejemplo, obtenemos de un texto los tokens que corresponden a cantidade de dinero (NER) y, utilizando relaciones de dependencias, obtnemos a qué concepto corresponden cada una de las entidades

In [ ]:
nlp_en = spacy.load("en_core_web_sm")
# Merge noun phrases and entities for easier analysis
nlp_en.add_pipe("merge_entities")
nlp_en.add_pipe("merge_noun_chunks")

TEXTS = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]
for doc in nlp_en.pipe(TEXTS):
    for token in doc:
        if token.ent_type_ == "MONEY":
            
            # We have an attribute and direct object, so check for subject
            if token.dep_ in ("attr", "dobj"):
                subj = [w for w in token.head.lefts if w.dep_ == "nsubj"]
                if subj:
                    print(subj[0], "-->", token)

            # We have a prepositional object with a preposition
            elif token.dep_ == "pobj" and token.head.dep_ == "prep":
                print(token.head.head, "-->", token)

## Similitud semántica entre palabras, frases y documentos

spaCy permite [calcular la similitud semántica](https://spacy.io/usage/vectors-similarity) entre cualquier par de objetos de tipo `Doc`, `Span` o `Token`. 

Ojo, la similitud semántica es un concepto algo subjetivo, pero en este caso se puede entender como la probabilidad de que dos palabras aparezcan en los mismos contextos. Vermos más sobre esto en otros notebooks

In [ ]:
# cargamos el modelo entrenado en inglés
nlp_en = spacy.load('en_core_web_md')

In [ ]:
# analizamos algunas colocaciones en inglés
token1, _, token2 = nlp_en("cats and dogs")
token3, _, token4 = nlp_en("research and development")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

In [ ]:
# ¿qué tal funciona en español?
token1, _, token2 = nlp_es("perros y gatos")
token3, _, token4 = nlp_es("investigación y desarrollo")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))